# Road Traffic Vehicle Counting Exam

In this exam, you will use the YOLO (You Only Look Once) object detection model to analyze a video of road traffic. Your primary task is to count the number of vehicles exiting the road at each of the four exits (located at the top, bottom, left, and right of the frame). The video for this task is provided at `Datasets/YOLO_Exam_Video.mp4`.

## Objectives
- Load and process the video using OpenCV.
- Utilize the YOLO model to detect vehicles in each frame.
- Manipulate the video frames using OpenCV.
- Track vehicles as they move across the video frames.
- Count the number of vehicles exiting via the top street.
- Count the number of vehicles exiting via the bottom street.
- Count the number of vehicles exiting via the left street.
- Count the number of vehicles exiting via the right street.


## Setup Environment

Before you begin, ensure you have the necessary libraries installed. You will need `opencv`, and `ultralytics` among others.
If these are not installed, you should install them.

In [1]:
!pip install ultralytics

In [2]:
import cv2

## Load the YOLO Model

In [3]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/My Drive/T5/week6'

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/T5/week6


You will first need to load the YOLO model, there is a custom model for this exam called `YOLO_Model.pt` under Datasets file ⚠️ PLEASE USE THIS MODEL AND DO NOT USE ANY OTHER MODEL ⚠️.
Write the code to load the YOLO model below:

In [4]:
from ultralytics import YOLO

model_path = '/content/drive/MyDrive/T5/week6/yolov8n.pt'
model = YOLO(model_path)

## Prepare the Video Capture

Create a variable to capture the video frames, you can use `cv2.VideoCapture()` to achive this.

In [5]:
video_path = '/content/drive/MyDrive/T5/week6/YOLO_Video.mp4'
cap = cv2.VideoCapture(video_path)

## Get Video Information

You can use `cv2` library to get these information fro the `VideoCapture()` variable you created to extract these information:
* `height`: Video's height.
* `width`: Video's width.
* `fps`: Video's frames.

In [6]:
hight = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
fps = int(cap.get(cv2.CAP_PROP_FPS))

In [7]:
width

1920

In [8]:
hight

1080

## Prepare Video Writer to Store the Output

Create a variable that uses `cv2.VideoCapture()` to save the video with the bounding boxes, specified Region of Interest rectangle (ROI) and the counted vehicles on each sides. You will need to make the video with the same `fps`, `width`, `height`, and specify the codec and output path of the video.

In [9]:
output_path = '/content/output_video_test.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (width, hight))

## Expermint to get the `(x1, y1, x2, y2)` of the ROI

### Hint
When defining the Regions of Interest (ROIs) for detecting vehicles exiting the road, consider the coordinates that outline the relevant areas. These coordinates can be specified as `(x1, y1, x2, y2)`, where `(x1, y1)` is the top-left corner and `(x2, y2)` is the bottom-right corner of the rectangle.

Experiment with different coordinates to accurately cover the streets where vehicles exit. Visualize these ROIs by drawing rectangles on a frame of the video and adjust the coordinates as needed until you capture the desired areas effectively.

In [10]:
# i define it in the loop down

In [11]:
ret, frame = cap.read()

In [12]:
frame

array([[[ 41,  77,  72],
        [ 54,  90,  85],
        [ 38,  79,  73],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[ 22,  58,  53],
        [ 49,  85,  80],
        [ 33,  74,  68],
        ...,
        [ 56,  52,  48],
        [ 55,  51,  47],
        [ 55,  51,  47]],

       [[  6,  42,  37],
        [ 40,  76,  71],
        [ 34,  75,  69],
        ...,
        [ 55,  51,  47],
        [ 55,  51,  47],
        [ 54,  50,  46]],

       ...,

       [[128, 118, 194],
        [111, 101, 177],
        [ 99,  87, 163],
        ...,
        [108, 117, 123],
        [108, 117, 123],
        [108, 117, 123]],

       [[117, 100, 183],
        [101,  84, 167],
        [112, 100, 164],
        ...,
        [110, 119, 125],
        [108, 117, 123],
        [108, 117, 123]],

       [[ 99,  82, 165],
        [106,  89, 172],
        [126, 114, 178],
        ...,
        [110, 119, 125],
        [110, 119, 125],
        [110, 119, 125]]

In [13]:
color = (0, 255, 0)
thickness = 3
font = cv2.FONT_HERSHEY_SIMPLEX

In [14]:
leftside = set()
rightside = set()
topside = set()
bottomside = set()

def check(center, id):
    if 800 < center[0] < 1400 and 500 < center[1] < 600:
        rightside.add(str(id))
    elif 1500 < center[0] < 1800 and 600 < center[1] < 900:
        leftside.add(str(id))
    elif 800 < center[0] < 1300 and 600 < center[1] < 900:
        topside.add(str(id))
    elif 1500 < center[0] < 1800 and 600 < center[1] < 900:
        bottomside.add(str(id))

In [16]:
RIO = [
    (500, 300, 630, 450),
    (1300, 500, 1400, 800),
    (700, 900, 800, 1100),
    (1100, 130, 1300, 250)
]

name = "lama almarri"

ret = True
while ret:
    ret, frame = cap.read()

    if not ret:
        break

    results = model.track(frame, persist=True)

    for result in results:
        for obj in result.boxes:
            bbox = obj.xyxy[0].cpu().numpy()
            class_id = int(obj.cls[0].cpu().numpy()) if obj.cls is not None else -1
            conf = obj.conf[0].cpu().numpy() if obj.conf is not None else 0.0
            id = int(obj.id[0].cpu().numpy()) if obj.id is not None else -1

            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, thickness)

            x_center = (x2 + x1) // 2
            y_center = (y1 + y2) // 2
            center = [x_center, y_center]
            check(center, id)

            label1 = f'ID: {id} Class: {class_id} Conf: {conf:.2f}'
            cv2.putText(frame, label1, (x1, y1 - 10), font, 0.5, color, 1, cv2.LINE_AA)

            label2 = f'Left side: {len(leftside)} Right side: {len(rightside)} Top side: {len(topside)} Bottom side: {len(bottomside)}'
            cv2.putText(frame, label2, (10, hight - 10), font, 0.5, color, 1, cv2.FONT_HERSHEY_SIMPLEX)
            cv2.putText(frame, name, (10, 50), font, 1, color, 2, cv2.LINE_AA)

    out.write(frame)

cap.release()
out.release()


## Process Video Frames and Identify Vehicles Left the Road Form Each Street

For each frame in the video, use the YOLO model to detect and track vehicles. You will need to write a loop that processes each frame and applies the YOLO model.

In each frame, after detecting the vehicles, determine whether they exited the road via the top, bottom, left, or right street. You can use the positions of the bounding boxes provided by YOLO to do this.

- The video should display bounding boxes around the detected objects.
- The video should display your name on the top.
- It should display the calculated center of each vehicle.
- The video should display the confidence score, alongside the object ID and class ID of each detected and tracked object.
- The video should display the rectangles representing the regions where you count the vehicles that have crossed and exited.
- The video should display the number of vehicles that exited via the top street.
- The video should display the number of vehicles that exited via the bottom street.
- The video should display the number of vehicles that exited via the left street.
- The video should display the number of vehicles that exited via the right street.

## Save and Submit Your Work

Submit both the notebook and the output video

In [ ]:
https://drive.google.com/file/d/1fdlnWGKT9wzHOaqAyAph3aWoyISDYDYy/view?usp=sharing